In [96]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import datetime

In [100]:
#assign url or website for our source to scrape.
url = "https://en.wikipedia.org/wiki/Nasdaq-100"
#grab the content of URL page
page = requests.get(url)
#use beautifulsoup to navigate url and extract data
soup = BeautifulSoup(page.content, "html.parser")
#searching all tag element specific for 'table'
table = soup.find('table', class_ ="wikitable sortable", id="constituents" )

In [101]:
#create new empty list 
Company = []
Tickers = []
Sector = []
Sub_Industry = []


#using for loop to find all data inside tag 'tr' or table row 
for row in table.findAll('tr')[1:]:
    # find all "td" from "tr"
    Name = row.findAll('td')[0].text
    Symbol = row.findAll('td')[1].text
    Sectors = row.findAll('td')[2].text
    Subsectors = row.findAll('td')[3].text
    
    Company.append(Name.rstrip("\n"))
    Tickers.append(Symbol.rstrip("\n"))
    Sector.append(Sectors.rstrip("\n"))
    Sub_Industry.append(Subsectors.rstrip("\n"))



In [102]:
#create data frame using pandas and assign each data frame with every list that content data that we scrap from wikipedia
df=pd.DataFrame()
df['Company']= Company
df['Ticker']= Tickers
df['Sector']= Sector
df['Sub_Industry']= Sub_Industry
df

,Company,Ticker,Sector,Sub_Industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe Inc.,ADBE,Information Technology,Application Software
2,Advanced Micro Devices,AMD,Information Technology,Semiconductors
3,Alexion Pharmaceuticals,ALXN,Health Care,Pharmaceuticals
4,Align Technology,ALGN,Health Care,Health Care Supplies
...,...,...,...,...
97,"Walgreen Boots Alliance, Inc.",WBA,Consumer Staples,Drug Retail
98,"Workday, Inc.",WDAY,Information Technology,Application Software
99,Xcel Energy Inc.,XEL,Utilities,Multi-Utilities
100,"Xilinx, Inc.",XLNX,Information Technology,Semiconductors


In [104]:
#sort 100 company that list in nasdaq to specific sector in information technology

newdf = df.loc[(df.Sector == "Information Technology")]

In [107]:
newdf.shape

(41, 4)

In [108]:
newdf

,Company,Ticker,Sector,Sub_Industry
1,Adobe Inc.,ADBE,Information Technology,Application Software
2,Advanced Micro Devices,AMD,Information Technology,Semiconductors
10,Analog Devices,ADI,Information Technology,Semiconductors
11,Ansys,ANSS,Information Technology,Application Software
12,Apple Inc.,AAPL,Information Technology,"Technology Hardware, Storage & Peripherals"
13,"Applied Materials, Inc.",AMAT,Information Technology,Semiconductor Equipment
14,ASML Holding,ASML,Information Technology,Semiconductor Equipment
15,Atlassian,TEAM,Information Technology,Application Software
16,Autodesk,ADSK,Information Technology,Application Software
17,"Automatic Data Processing, Inc.",ADP,Information Technology,Data Processing & Outsourced Services


In [111]:
# I created own key from alphaa vantage website
key = "G6QD9PVL6NQM5KIJ"
tickers = newdf["Ticker"].to_list()
# create empty list to store all data 
Open = []
High = []
Low = []
Close = []
Volume = []
# count variable for time function. see below explaination
count = 0

# create for loop to generate api each ticker in tickers ( Stock list ) 
for ticker in tickers:
    # since alpha vantage only allow us to request API 5 times every 1 minute or 500 times for one day, I let the python to sleep with time function
    # every time the program run 5 times API, it will sleep for 60 second and it will continue to run the rest list.
    count = count + 1
    if count % 5 == 0:
        time.sleep(60)
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol="+ticker+"&apikey="+key
    #print(ticker)
    r = requests.get(url)
    if(r.status_code == 200):
        # Scrap and append data from json to the list
        data = r.json()
        #print(json.dumps(data, sort_keys=True, indent=10))
        Open.append(float(data["Time Series (Daily)"]["2021-05-19"]["1. open"]))
        #print("Open:"+ str(data["Time Series (Daily)"]["2021-05-19"]["1. open"]))
        
        High.append(float(data["Time Series (Daily)"]["2021-05-19"]["2. high"]))
        #print("High:"+ str(data["Time Series (Daily)"]["2021-05-19"]["2. high"]))
        
        Low.append(float(data["Time Series (Daily)"]["2021-05-19"]["3. low"]))
        #print("Low:"+ str(data["Time Series (Daily)"]["2021-05-19"]["3. low"]))
        
        Close.append(float(data["Time Series (Daily)"]["2021-05-19"]["4. close"]))
        #print("Close:"+ str(data["Time Series (Daily)"]["2021-05-19"]["4. close"])
        
        Volume.append(float(data["Time Series (Daily)"]["2021-05-19"]["5. volume"]))
        #print("Volume:"+ str(data["Time Series (Daily)"]["2021-05-19"]["5. volume"]))

In [143]:
Stock = {"Open" : Open,
         "High" : High,
         "Low" : Low,
         "Close" : Close,
         "Volume" : Volume}

df1 = pd.DataFrame(Stock)

In [147]:
newdf.reset_index(drop=True, inplace=True)
df1.reset_index(drop=True, inplace=True)

In [152]:
dfjoin = pd.concat([newdf,df1], axis = 1)
dfjoin

,Company,Ticker,Sector,Sub_Industry,Open,High,Low,Close,Volume
0,Adobe Inc.,ADBE,Information Technology,Application Software,473.0200,480.870,472.0000,480.47,2047511.0
1,Advanced Micro Devices,AMD,Information Technology,Semiconductors,73.1600,76.970,72.7600,76.23,73966162.0
2,Analog Devices,ADI,Information Technology,Semiconductors,145.7900,153.330,144.0700,153.12,7066133.0
3,Ansys,ANSS,Information Technology,Application Software,316.2400,327.320,316.2200,326.49,326358.0
4,Apple Inc.,AAPL,Information Technology,"Technology Hardware, Storage & Peripherals",123.1600,124.915,122.8600,124.69,92611989.0
5,"Applied Materials, Inc.",AMAT,Information Technology,Semiconductor Equipment,118.4200,125.080,117.7900,124.80,10424174.0
6,ASML Holding,ASML,Information Technology,Semiconductor Equipment,608.0100,630.980,606.6400,630.78,1049576.0
7,Atlassian,TEAM,Information Technology,Application Software,211.9100,216.580,209.2750,216.18,1777497.0
8,Autodesk,ADSK,Information Technology,Application Software,267.6300,274.710,266.7901,273.97,821251.0
9,"Automatic Data Processing, Inc.",ADP,Information Technology,Data Processing & Outsourced Services,190.3200,191.990,187.2900,190.84,1668151.0


In [153]:
dfjoin.to_csv('Stock price 2021-05-19')

In [154]:
dfjoin.describe()

,Open,High,Low,Close,Volume
count,41.000000,41.000000,41.000000,41.000000,4.100000e+01
mean,210.284080,217.467805,209.352590,217.016585,9.375765e+06
std,144.084696,149.905441,143.774042,149.806871,1.867257e+07
min,41.460000,44.830000,41.250000,44.510000,3.263580e+05
25%,114.710000,117.130000,114.250000,117.070000,1.576855e+06
50%,175.240000,181.930000,175.000000,181.720000,2.047511e+06
75%,239.310000,245.340000,238.600000,244.630000,7.210456e+06
max,608.010000,630.980000,606.640000,630.780000,9.261199e+07
